<a href="https://colab.research.google.com/github/sunnysavita10/Generative-AI-Indepth-Basic-to-Advance/blob/main/RAG_with_Conversation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma beautifulsoup4
!pip install -q langchain_google_genai


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "RAG_With_Memory"

In [25]:
pip install google.colab


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement google.colab (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for google.colab


In [30]:
import os

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")  # reads from system env
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


In [31]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [33]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-1.0-pro",convert_system_message_to_human=True)

In [34]:
print(model.invoke("hi").content)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised NotFound: 404 models/gemini-1.0-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised NotFound: 404 models/gemini-1.0-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised NotFound: 404 models/gemini-1.0-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._chat_with_retry.<local

NotFound: 404 models/gemini-1.0-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

In [35]:
import bs4
from langchain import hub

In [36]:
from langchain.chains import create_retrieval_chain

In [37]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [38]:
from langchain_chroma import Chroma

In [39]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [40]:
from langchain_core.prompts import ChatPromptTemplate

In [41]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [42]:
from langchain_core.prompts import MessagesPlaceholder

In [43]:
loader = WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))),)

In [44]:
doc = loader.load()

In [45]:
doc

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [46]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(doc)

In [47]:
vectorstore = Chroma.from_documents(documents=splits, embedding=gemini_embeddings)
retriever = vectorstore.as_retriever()

In [48]:
retriever

VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002C62D3D6330>, search_kwargs={})

In [49]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question "
    "If you don't know the answer, say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

In [50]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [51]:
question_answering_chain=create_stuff_documents_chain(model, chat_prompt)

In [52]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [53]:
response = rag_chain.invoke({"input":"what is MRKL?"})

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised NotFound: 404 models/gemini-1.0-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised NotFound: 404 models/gemini-1.0-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised NotFound: 404 models/gemini-1.0-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._chat_with_retry.<local

NotFound: 404 models/gemini-1.0-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

In [54]:
response["answer"]

NameError: name 'response' is not defined

In [ ]:
from langchain.chains import create_history_aware_retriever


In [ ]:
retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)

In [ ]:
contextualize_q_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),


     ]
)

In [ ]:
history_aware_retriever = create_history_aware_retriever(model,retriever,contextualize_q_prompt)

In [ ]:
from langchain.chains import create_retrieval_chain

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

In [ ]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

In [ ]:
chat_history = []

In [ ]:
question1 = "what is Task Decomposition?"

In [ ]:
message1= rag_chain.invoke({"input": question1, "chat_history": chat_history})

In [ ]:
message1["answer"]

'Task decomposition is a technique that breaks down complex tasks into smaller, more manageable steps. This can be done using various methods, such as LLM prompting, task-specific instructions, or human input. Task decomposition helps agents plan ahead and execute tasks more efficiently.'

In [ ]:
chat_history.extend(
    [
        HumanMessage(content=question1),
        AIMessage(content=message1["answer"]),
    ]
)

In [ ]:
chat_history


[HumanMessage(content='what is Task Decomposition?'),
 AIMessage(content='Task decomposition is a technique that breaks down complex tasks into smaller, more manageable steps. This can be done using various methods, such as LLM prompting, task-specific instructions, or human input. Task decomposition helps agents plan ahead and execute tasks more efficiently.')]

In [ ]:
second_question = "What are common ways of doing it?"
message2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(message2["answer"])

Common ways of doing task decomposition include:

1. **LLM prompting:** Instructing the LLM to "think step by step" or "decompose the task into smaller steps."
2. **Task-specific instructions:** Providing the LLM with specific instructions on how to break down the task, such as "Write a story outline" for writing a novel.
3. **Human input:** Manually decomposing the task into smaller steps and providing the LLM with the decomposed task.


In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
store = {}

In [ ]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task decomposition involves breaking down a complex task into smaller, manageable steps. It can be done through LLM prompting techniques, task-specific instructions, or human inputs. The purpose of task decomposition is to make complex tasks more manageable for AI agents.'

In [ ]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?'), AIMessage(content='Task decomposition involves breaking down a complex task into smaller, manageable steps. It can be done through LLM prompting techniques, task-specific instructions, or human inputs. The purpose of task decomposition is to make complex tasks more manageable for AI agents.')])}

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Common ways of doing task decomposition include using LLM prompting techniques like Chain of Thought (CoT) or Tree of Thoughts (ToT), providing task-specific instructions, or obtaining human inputs to break down the task into smaller steps.'

In [ ]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Task Decomposition?

AI: Task decomposition involves breaking down a complex task into smaller, manageable steps. It can be done through LLM prompting techniques, task-specific instructions, or human inputs. The purpose of task decomposition is to make complex tasks more manageable for AI agents.

User: What are common ways of doing it?

AI: Common ways of doing task decomposition include using LLM prompting techniques like Chain of Thought (CoT) or Tree of Thoughts (ToT), providing task-specific instructions, or obtaining human inputs to break down the task into smaller steps.



In [ ]:
conversational_rag_chain.invoke(
    {"input": "What is a prompt technique like step xyz?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'A prompt technique like "Step XYZ" is a type of instruction given to a language model (LM) or other AI system to guide it in performing a task. It typically involves breaking down a complex task into a series of smaller, more manageable steps. The LM is then prompted to execute each step in sequence, with the output of one step becoming the input for the next. This technique can help the LM to reason through complex problems and generate more accurate and coherent responses.'

In [ ]:
store

In [ ]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Task Decomposition?

AI: Task decomposition involves breaking down a complex task into smaller, manageable steps. It can be done through LLM prompting techniques, task-specific instructions, or human inputs. The purpose of task decomposition is to make complex tasks more manageable for AI agents.

User: What are common ways of doing it?

AI: Common ways of doing task decomposition include using LLM prompting techniques like Chain of Thought (CoT) or Tree of Thoughts (ToT), providing task-specific instructions, or obtaining human inputs to break down the task into smaller steps.

User: What is a prompt technique like step xyz?

AI: A prompt technique like "Step XYZ" is a type of instruction given to a language model (LM) or other AI system to guide it in performing a task. It typically involves breaking down a complex task into a series of smaller, more manageable steps. The LM is then prompted to execute each step in sequence, with the output of one step becoming the inpu